### Hedging: Volatility Mismatch

Consider a short position in a European call option on a non-dividend paying stock with a maturity of one year and strike K = 99 EUR. Let the one-year risk-free interest rate be 6% and the current stock price be 100 EUR. Furthermore, assume that the volatility is 20%.

Use the Euler method to perform a hedging simulation.

In [2]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

### Model Construction

#### 1. Matching Volatility

Conduct an experiment where the volatility in the stock price process matches the volatility used in the delta computation (i.e., both set to 20%). Vary the frequency of hedge adjustments (from daily to weekly) and explain the results.

In [1]:
# Parameters

S0 = 100 # Spot price
K = 99 # strike price
T = 1.0 # maturity
r = 0.06 # risk-free rate
sigma = 0.20 # volatility (real same as implied for task 1)

def get_call_price(S, t, K, T, r, sigma):
    """Black-Scholes European call price (eq. 91)"""
    tau = T - t
    if tau <= 0:
        return max(S - K, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    return S * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)

def get_delta(S, t, K, T, r, sigma):
    """Black Scholes delta = N(d1) (eq.97 + Greeks table)"""
    tau = T - t
    if tau <= 0:
        return 1.0 if S > K else 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    return norm.cdf(d1)
    

In [4]:
def simulate_hedge(S0, T, K, r, sigma_r, sigma_i, N_steps, seed = None):
    """
    Simulate one delta-hedge path.
    
    sigma_r: volatility driving the stock
    sigma_i: volatility used for option pricing and delta calculation (implied volatility)
    
    """
    
    if seed is not None:
        np.random.seed(seed)
        
    dt = T / N_steps # time step
    S = np.zeros(N_steps + 1)
    S[0] = S0 # set spot price
    
    # Simulate stock path via Euler scheme
    Z = np.random.randn(N_steps)
    for i in range(N_steps):
        S[i+1] = S[i] + r * S[i] * dt + sigma_r * S[i] * np.sqrt(dt) * Z[i] # standard EM scheme for GBM
        
    # Init hedge
    delta_0 = get_delta(S[0], 0, K, T, r, sigma_i) # Initial delta based on the initial price and time
    C0 = get_call_price(S[0], 0, K, T, r, sigma_i) # Initial option price
    B = C0 - delta_0 * S[0] # Bank acccount: initial option price - initial stock position
    delta_old = delta_0
    
    # Rebalance at each step
    for i in range(1, N_steps):
        t_i = i * dt
        B = B * np.exp(r * dt) # Bank account grows at risk-free rate
        delta_new = get_delta(S[i], t_i, K, T, r, sigma_i) # New delta based on current price and time
        B = B - (delta_new - delta_old) * S[i]           # Rebalance: adjust bank account by the cost of changing the stock position
        delta_old = delta_new
        
    # Final PnL at maturity
    B = B * np.exp(r * dt)  
    payoff = max(S[-1] - K, 0)   # Option settlement
    PnL = B + delta_old * S[-1] - payoff # Total PnL: bank account + final stock position - option payoff
    
    return PnL, S

In [7]:
# Monte Carlo across different rebalancing frequnecies
n_sims = 10_000
frequencies = {                 # NOTE: We can change these frequnecies
    "Daily (N=252)": 252,
    "Every 2 days (N=126)": 126,
    "Weekly (N=52)": 52,
    "Biweekly (N=26)": 26,
}

sigma_i = 0.20
sigma_r = 0.20

results = {}
for label, N in frequencies.items():
    pnls = []
    for sim in range(n_sims):
        pnl, _ = simulate_hedge(S0, T, K, r, sigma_r, sigma_i, N, seed=sim)
        pnls.append(pnl)
    results[label] = np.array(pnls)
    
    print(f"{label}:")
    print(f"  Mean PnL:  {np.mean(pnls):.4f}")
    print(f"  Std PnL:   {np.std(pnls):.4f}")
    print()

Daily (N=252):
  Mean PnL:  -0.0054
  Std PnL:   0.4267

Every 2 days (N=126):
  Mean PnL:  -0.0106
  Std PnL:   0.6030

Weekly (N=52):
  Mean PnL:  -0.0020
  Std PnL:   0.9358

Biweekly (N=26):
  Mean PnL:  -0.0038
  Std PnL:   1.3136



#### Interpretation

---

#### 2. Mismatched Volatility

Perform numerical experiments where the volatility in the stock price process does not match the volatility used in the delta valuation. Run computational experiments for various levels of volatility and discuss the outcomes.

In [10]:
# Task 2: Mismatched Volatility
sigma_imp = 0.20
sigma_real_grid = [0.10, 0.15, 0.20, 0.25, 0.30]
N_steps = 252
n_simulations = 10_000


results_mismatch = {}
for sigma_real in sigma_real_grid:
    pnls = []
    for sim in range(n_simulations):
        pnl, _ = simulate_hedge(S0, T, K, r, sigma_real, sigma_imp, N_steps, seed=sim)
        pnls.append(pnl)
    results_mismatch[sigma_real] = np.array(pnls)
    print(f"σ_real = {sigma_real:.2f}:")
    print(f"  Mean PnL:  {np.mean(pnls):.4f}")
    print(f"  Std PnL:   {np.std(pnls):.4f}")

σ_real = 0.10:
  Mean PnL:  3.5862
  Std PnL:   1.1181
σ_real = 0.15:
  Mean PnL:  1.8635
  Std PnL:   0.7369
σ_real = 0.20:
  Mean PnL:  -0.0054
  Std PnL:   0.4267
σ_real = 0.25:
  Mean PnL:  -1.9297
  Std PnL:   1.0024
σ_real = 0.30:
  Mean PnL:  -3.8816
  Std PnL:   1.9202


#### Interpretation

---

#### 3. Pricing and Hedging with Implied Volatility

Core Idea:

1. We price the call option price and delta using the implied volatilty
2. The stock follows the realized volatility
3. In order to create a risk-free replicating portfolio at any time instant we must have:

$$

d\Pi_t = dV_t - dC_t 

$$

where $V_t$ the change in the portfolio value and $C_t$ the change in the option value.

Assuming the stock price follows (under the real world measure $P$), the SDE is given by

$$

dS_t = \mu S_t dt + \sigma_{real} S_t dW_t

$$

where $\sigma_{real}$ the realized volatility.

Since the portfolio is self-financing, its change in value due to stock interest is

$$

d\Pi_t = \left[ \Delta dS_t + r(V_t - \Delta S_t) dt \right] - \left[ \frac{\partial C}{\partial t} dt + \frac{\partial C}{\partial S} dS_t + \frac{1}{2} \sigma^2_{real} S_t^2 \Gamma dt \right]

$$